In [12]:
# Insert all parameters here. Note that this is the only place where a user inputs can be processed.

# Altergo parameters
factory_api = "https://altergo.io/"
iot_api = "https://iot.altergo.io/" 
api_key = "" # Note that your API key will become public when it is injected

# Setup parameters
asset_sn = "ISO-ENTSO-E_Belgium_BE" # Get the target input ISO asset serial number
target_asset = ["PRED-ALFN-BANK-001-01","PRED-ALFN-BANK-001-02"] #Get the target list of assets 
input_sn_list = ["DALMP"] #Get the input sensor list
sn_list = ["SOC_dispatch","E_out_DAM","E_in_DAM","Revenue Generated","Total Revenue","Charge Cost","Profit Sum","Revenue Sum"] # Get the target list of sensors
start_time = "2021-9-1 00:00" # Get the start time for calculating: Expect as YYYY-MM-DD hh:mm:ss
end_time = "2021-9-8 00:00" # Get the end time for calculating : Expect as YYYY-MM-DD hh:mm:ss

# Model Parameters
initial_SoC = "50" # Get the model parameter

# Update flag
should_update = False

In [13]:
from IPython.display import clear_output
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import json
from scipy import stats
from typing import Union
from copy import deepcopy
from getpass import getpass

In [14]:
# Input cleanup and setup

os.environ["ALTERGO_FACTORY_API"] = factory_api
os.environ["ALTERGO_IOT_API"] = iot_api

start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
start_time = start_time.timestamp() * 1000
end_time = end_time.timestamp() * 1000

In [22]:

!brew install -qqq -U pyomo
!pip install glpk
!pip install -qqq git+https://bitbucket.org/freemens/ion_sdk.git@master
clear_output()


In [ ]:
from pyomo.environ import *
from ion_sdk.edison_api.models.factoryModel import EdisonGenericComponent, Model,CurrentState
from ion_sdk.edison_api.edison_api import Client, dataUpdateMethod, edisonDate
from ion_sdk.edison_api.models.factory import getParameterValue, getSensorCodeByName, EdisonComponent
from ion_sdk.tools.toolbox import decimate_data

In [ ]:
edApi = Client(api_key) # Connect to the ALTG Client
asset = edApi.getAsset(asset_sn) # Get the Asset from ALTG
asset1 = edApi.getAsset(target_asset[0])
asset2 = edApi.getAsset(target_asset[1])

edApi.getAssetDataFrame([asset], input_sn_list, start_time, end_time) # Get the asset dataframe from ALTG
energyCapacity = getParameterValue(asset1.model,'Energy','Nom') # Get the capacity of the battery
energyCapacity = 616.0
pmax = getParameterValue(asset1.model,'Energy','Nom') # Get the capacity of the battery
pmax = 308.0
nCycle =getParameterValue(asset1.model,'nCycles per day','Nom')
nCycle = 0.35
eff =90.0/100.0

In [ ]:
# This cell will vary from user to user, client to client
# Add any other adjustment steps in this cell
initial_SoC = int(initial_SoC)
asset.df.reset_index(inplace=True)
asset.df['time_stamp'] = asset.df.date
asset.df['time_stamp'] = pd.to_datetime(asset.df['time_stamp'])

for i in range(0,167):
   asset.df["time_stamp"].iloc[i] = asset.df["time_stamp"].iloc[i].replace(second=0, microsecond=0, minute=0, hour=asset.df["time_stamp"].iloc[i].hour)+timedelta(hours=asset.df["time_stamp"].iloc[i].minute//30)

asset.df['hour'] = asset.df.time_stamp.index
asset.df['Unnamed1: 0'] = asset.df['hour']
asset.df.insert(0, 'Unnamed: 0', asset.df['Unnamed1: 0'] )
asset.df.drop(["Unnamed1: 0","time_stamp"],axis=1,inplace=True)
asset.df = asset.df.rename(columns={'date': 'time_stamp','DALMP': 'lbmp'})
df1 = asset.df
df2 = asset.df

In [ ]:
# Set up the models
#@title 7 Day Average (Run time: < 5s)
def dispatch_strategy_1(df, energy_capacity, p_max, n_cycles, eff, initial_Soc, soc_low, soc_high):
    """
    Strategy 1: Take a mean for the day ahead. If price is higher than average, 
    discharge. If price is lower than average, charge. Do this while satisfying 
    all other constraints (like number of cycles in a day, SOC constraints, 
    etc.)

    """

    average_price = df["lbmp"].mean()
    df = df.reset_index()
    df["SOC"] = 0
    df.loc[0, "SOC"] = initial_Soc
    df["E_in"] = 0
    df["E_out"] = 0

    total_throughput = 0
    max_throughput = energy_capacity * n_cycles

    for t in range(1, df.index.size):
        if t % 24 == 0:
            total_throughput = 0
        
        df.loc[t, "SOC"] = df.loc[t-1,"SOC"]
        if df.loc[t,"lbmp"] > average_price:
            if df.loc[t-1, "SOC"] - p_max/eff/energy_capacity >= soc_low:
                if total_throughput < max_throughput:
                    df.loc[t,"E_out"] = p_max
                    df.loc[t, "SOC"] = df.loc[t-1, "SOC"] - df.loc[t,"E_out"]/eff/energy_capacity
                    total_throughput += p_max
            elif df.loc[t-1, "SOC"] - p_max/eff/energy_capacity/2 >= soc_low:
                if total_throughput < max_throughput:
                    df.loc[t,"E_out"] = p_max/2
                    df.loc[t, "SOC"] = df.loc[t-1, "SOC"] - df.loc[t,"E_out"]/eff/energy_capacity
                    total_throughput += p_max/2

        else:
            if df.loc[t-1,"SOC"] + p_max * eff / energy_capacity <= soc_high:
                df.loc[t,"E_in"] = p_max
                df.loc[t, "SOC"] = df.loc[t-1, "SOC"] + df.loc[t,"E_in"] * eff / energy_capacity

    return df.set_index("time_stamp")

asset1.df = dispatch_strategy_1(df1, energyCapacity, pmax, nCycle, eff, (initial_SoC/100.0), 0.05, 0.95)

In [ ]:
#@title Hourly Average (Run time: < 5s)
def dispatch_strategy_2(df, energy_capacity, p_max, n_cycles, eff, initial_Soc, soc_low, soc_high):
    """
    Take average across 7 days for each hour. If the price for that hour is 
    greater than the average price for that hour, discharge. If the price for 
    that hour is less than the average price for that hour, charge. Also ensure 
    that all other constraints are met (throughput, SOC, etc.)
    """

    df = df.reset_index()
    df["SOC"] = 0
    df.loc[0, "SOC"] = initial_Soc
    df["E_in"] = 0
    df["E_out"] = 0

    total_throughput = 0
    max_throughput = energy_capacity * n_cycles

    df.index = df.index % 24 # Convert to 24 hour indices. Makes it easier for taking averages.
    average_price_df = pd.DataFrame({"Average LMP" : [0 for _ in range(0, 24)]}, index = range(0, 24))
    for i in range(0, 24):
        average_price_df.loc[i, "Average LMP"] = df.loc[i,"lbmp"].mean()

    df = df.reset_index(drop = True)
    for t in range(1, df.index.size):
        if t % 24 == 0:
            total_throughput = 0

        df.loc[t, "SOC"] = df.loc[t-1,"SOC"]
        if df.loc[t,"lbmp"] > average_price_df.loc[t % 24, "Average LMP"]:
            if df.loc[t-1, "SOC"] - p_max / eff / energy_capacity >= soc_low:
                if total_throughput < max_throughput:
                    df.loc[t,"E_out"] = p_max
                    df.loc[t, "SOC"] = df.loc[t-1, "SOC"] - df.loc[t,"E_out"]/eff/energy_capacity
                    total_throughput += p_max
            elif df.loc[t-1, "SOC"] - p_max/eff/energy_capacity/2 >= soc_low:
                if total_throughput < max_throughput:
                    df.loc[t,"E_out"] = p_max/2
                    df.loc[t, "SOC"] = df.loc[t-1, "SOC"] - df.loc[t,"E_out"]/eff/energy_capacity
                    total_throughput += p_max/2

        else:
            if df.loc[t-1,"SOC"] + p_max * eff / energy_capacity <= soc_high:
                df.loc[t,"E_in"] = p_max
                df.loc[t, "SOC"] = df.loc[t-1, "SOC"] + df.loc[t,"E_in"] * eff / energy_capacity

    return df.set_index("time_stamp")

asset2.df = dispatch_strategy_2(df2, energyCapacity, pmax, nCycle, eff, (initial_SoC/100.0), 0.05, 0.95)

In [ ]:
asset1.df = asset1.df.rename(columns={'E_in': 'E_in_DAM', "E_out" : "E_out_DAM","SOC" : "SOC_dispatch"})
asset2.df = asset2.df.rename(columns = {'E_in': 'E_in_DAM', "E_out" : "E_out_DAM","SOC" : "SOC_dispatch"})

for x in [asset1, asset2]:
    x.df["Revenue Generated"] = x.df["E_out_DAM"] * x.df["lbmp"]
    x.df["Charge Cost"] = x.df["E_in_DAM"] * x.df["lbmp"]
    x.df["Total Revenue"] = x.df["Revenue Generated"] - x.df["Charge Cost"]
    x.df["Revenue Sum"] = x.df["Revenue Generated"].cumsum()
    x.df["Profit Sum"] = x.df["Total Revenue"].cumsum()

In [ ]:
# Return data to ALTG

# This cell will vary from user to user, client to client.
# Depending on the architecture, the client will have a set sensor to which data needs to be returns for State of Health    

edApi.updateSensorDataByFile(asset1, sn_list)
edApi.updateSensorDataByFile(asset2, sn_list)
print("Successfully Uploaded Data to Edison")

In [ ]:
'''{
   "factory_api": "https://altergo.io/",
   "iot_api": "https://iot.altergo.io/",
   "api_key": "",
   "asset_sn": "ISO-ENTSO-E_Belgium_BE",
   "target_asset" = ["PRED-ALFN-BANK-001-01","PRED-ALFN-BANK-001-02","PRED-ALFN-BANK-001-03"]
   "input_sn_list" = ["DALMP"] 
   "sn_list" = ["SOC_dispatch","E_out_DAM","E_in_DAM","Revenue Generated","Total Revenue","Charge Cost","Profit Sum","Revenue Sum"]
   "start_time" = "2021-9-1 00:00" 
   "end_time" = "2021-9-8 00:00" 
   "initial_SoC" = "50"
   "should_update": true,
 }'''